In [ ]:
# ⚙️ Install TensorFlow
!pip install tensorflow

# 📦 Load dependencies
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Embedding, MultiHeadAttention, LayerNormalization, Dropout, Dense, GlobalAveragePooling1D
from tensorflow.keras.models import Model

# 📥 Load data
df = pd.read_csv("/content/faulty_coral_dna_dataset_20k_corrected.csv")

# 🧬 Tokenize DNA
BASE5 = {'A': 0, 'C': 1, 'G': 2, 'T': 3, 'N': 4}
MAX_LEN = 300
df["tokens"] = df["Faulty_Sequence"].apply(lambda seq: [BASE5.get(b, 4) for b in str(seq)[:MAX_LEN]] + [4] * (MAX_LEN - len(seq)))
df["label"] = df["Bleaching_Status"].str.lower().map({"healthy": 1, "bleached": 0})

# ✂️ Prepare data
X = np.stack(df["tokens"].values)
y = df["label"].values.astype(np.int32)
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=0.2)

# 🧠 Transformer Encoder Block
def transformer_encoder(inputs, head_size=64, num_heads=4, ff_dim=128, dropout=0.1):
    # Multi-head attention block
    attention = MultiHeadAttention(num_heads=num_heads, key_dim=head_size)(inputs, inputs)
    attention = Dropout(dropout)(attention)
    x = LayerNormalization(epsilon=1e-6)(inputs + attention)

    # Feedforward block with projection to match input dim
    ffn = Dense(ff_dim, activation="relu")(x)
    ffn = Dropout(dropout)(ffn)
    ffn = Dense(inputs.shape[-1])(ffn)  # 🔧 Project back to embedding dim
    return LayerNormalization(epsilon=1e-6)(x + ffn)

# 🧱 Model
VOCAB_SIZE = 5
EMBED_DIM = 64

inputs = Input(shape=(MAX_LEN,), dtype=tf.int32)
x = Embedding(input_dim=VOCAB_SIZE, output_dim=EMBED_DIM)(inputs)
x = transformer_encoder(x)
x = transformer_encoder(x)
x = GlobalAveragePooling1D()(x)
x = Dense(64, activation="relu")(x)
x = Dropout(0.2)(x)
outputs = Dense(1, activation="sigmoid")(x)

model = Model(inputs, outputs)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

# 🏋️ Train
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)


Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_9       │ (None, 300)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_9         │ (None, 300, 64)   │        320 │ input_layer_9[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 300, 64)   │     66,368 │ embedding_9[0][0… │
│ (MultiHeadAttentio… │                   │            │ embedding_9[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_60          │ (None, 300, 64)   │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_35 (Add)        │ (None, 300, 64)   │          0 │ embedding_9[0][0… │
│                     │                   │            │ dropout_60[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 300, 64)   │        128 │ add_35[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_46 (Dense)    │ (None, 300, 128)  │      8,320 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_61          │ (None, 300, 128)  │          0 │ dense_46[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_47 (Dense)    │ (None, 300, 64)   │      8,256 │ dropout_61[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_36 (Add)        │ (None, 300, 64)   │          0 │ layer_normalizat… │
│                     │                   │            │ dense_47[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 300, 64)   │        128 │ add_36[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 300, 64)   │     66,368 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_63          │ (None, 300, 64)   │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_37 (Add)        │ (None, 300, 64)   │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_63[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 300, 64)   │        128 │ add_37[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_48 (Dense)    │ (None, 300, 128)  │      8,320 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_64          │ (None, 300, 128)  │          0 │ dense_48[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_49 (Dense)    │ (None, 300, 64)   │      8,256 │ dropout_64[0][0]

 Total params: 170,945 (667.75 KB)

 Trainable params: 170,945 (667.75 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 24s 24ms/step - accuracy: 0.6617 - loss: 0.6099 - val_accuracy: 0.8110 - val_loss: 0.4841
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 11s 23ms/step - accuracy: 0.8105 - loss: 0.4903 - val_accuracy: 0.8110 - val_loss: 0.4956
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 21s 23ms/step - accuracy: 0.8106 - loss: 0.4900 - val_accuracy: 0.8110 - val_loss: 0.4827
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 20s 23ms/step - accuracy: 0.8182 - loss: 0.4766 - val_accuracy: 0.8110 - val_loss: 0.4825
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 11s 23ms/step - accuracy: 0.8144 - loss: 0.4838 - val_accuracy: 0.8110 - val_loss: 0.4825
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 11s 23ms/step - accuracy: 0.8135 - loss: 0.4833 - val_accuracy: 0.8110 - val_loss: 0.4840
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 11s 23ms/step - accuracy: 0.8180 - loss: 0.4772 - val_accuracy: 0.8110 - val_loss: 0.4827
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 20s 23ms/step - accuracy: 0.8192 - loss: 0.4743 - 

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Predict probabilities and binary predictions
y_pred_proba = model.predict(X_val).flatten()
y_pred = (y_pred_proba >= 0.5).astype(int)

# Evaluation metrics
print("Accuracy:", accuracy_score(y_val, y_pred))
print("Precision:", precision_score(y_val, y_pred))
print("Recall:", recall_score(y_val, y_pred))
print("F1 Score:", f1_score(y_val, y_pred))
print("ROC AUC:", roc_auc_score(y_val, y_pred_proba))

# Plot training history
def plot_history(history):
    # Check for correct metric names
    metrics = history.history.keys()
    acc_key = "accuracy" if "accuracy" in metrics else "binary_accuracy"
    val_acc_key = "val_accuracy" if "val_accuracy" in metrics else "val_binary_accuracy"

    plt.figure(figsize=(12, 5))

    # Accuracy Plot
    plt.subplot(1, 2, 1)
    plt.plot(history.history[acc_key], label="Train Accuracy")
    plt.plot(history.history[val_acc_key], label="Validation Accuracy")
    plt.title("Model Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.grid(True)

    # Loss Plot
    plt.subplot(1, 2, 2)
    plt.plot(history.history["loss"], label="Train Loss")
    plt.plot(history.history["val_loss"], label="Validation Loss")
    plt.title("Model Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

# Plot Confusion Matrix
def plot_conf_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Bleached", "Healthy"])
    disp.plot(cmap=plt.cm.Blues)
    plt.title("Confusion Matrix")
    plt.show()

# Plot after training
plot_history(history)
plot_conf_matrix(y_val, y_pred)


In [ ]:
import pandas as pd
import numpy as np
import torch
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE
from pytorch_lightning import Trainer, seed_everything, LightningModule
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import math
import types

# ⚙️ Load dataset
df = pd.read_csv("/content/updated_scor_raw_df (22).csv")

# 🧼 Preprocess
agg_df = df.groupby(['SiteID', 'Year']).agg({
    'scor_lta': 'mean',
    'TempC': 'mean',
    'density': 'mean',
    'LTA_cm2': 'mean'
}).reset_index()

agg_df = agg_df.dropna(subset=['scor_lta'])

for col in ['TempC', 'density', 'LTA_cm2']:
    agg_df[col] = agg_df.groupby("SiteID")[col].transform(lambda x: x.fillna(x.mean()))
    agg_df[col] = agg_df[col].fillna(agg_df[col].mean())

agg_df['scor_lta'] = np.log1p(agg_df['scor_lta'])  # log transform
agg_df["time_idx"] = agg_df["Year"] - agg_df["Year"].min()
agg_df["SiteID"] = agg_df["SiteID"].astype(str)

# ⏳ Temporal parameters
max_encoder_length = 3
max_prediction_length = 1

site_counts = agg_df.groupby("SiteID")["Year"].nunique()
valid_sites = site_counts[site_counts >= (max_encoder_length + max_prediction_length)].index
agg_df = agg_df[agg_df["SiteID"].isin(valid_sites)]

# 📦 Dataset
training = TimeSeriesDataSet(
    agg_df,
    time_idx="time_idx",
    target="scor_lta",
    group_ids=["SiteID"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    static_categoricals=["SiteID"],
    time_varying_known_reals=["time_idx", "Year"],
    time_varying_unknown_reals=["scor_lta", "TempC", "density", "LTA_cm2"],
    target_normalizer=GroupNormalizer(groups=["SiteID"]),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True
)

# ✂️ Train/Validation split
val_cutoff = agg_df["time_idx"].max() - max_prediction_length
train_data = agg_df[agg_df["time_idx"] <= val_cutoff]
val_data = agg_df[agg_df["time_idx"] > val_cutoff]

def filter_min_years(df, min_years=4):
    site_counts = df.groupby("SiteID")["Year"].nunique()
    valid_sites = site_counts[site_counts >= min_years].index
    return df[df["SiteID"].isin(valid_sites)]

train_data = filter_min_years(train_data)
val_data = filter_min_years(val_data)

train_ds = TimeSeriesDataSet.from_dataset(training, train_data)
train_dataloader = train_ds.to_dataloader(train=True, batch_size=64, num_workers=0)

val_dataloader = None
if len(val_data) >= 1:
    val_ds = TimeSeriesDataSet.from_dataset(training, val_data)
    val_dataloader = val_ds.to_dataloader(train=False, batch_size=64, num_workers=0)
else:
    print("⚠️ Validation set is empty after filtering. Proceeding without validation.")

# ✅ LightningModule with metrics
class TFTLightningWrapper(LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.val_preds = []
        self.val_targets = []

    def training_step(self, batch, batch_idx):
        loss = self.model.training_step(batch, batch_idx)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.model.validation_step(batch, batch_idx)
        preds = self.model.predict(batch)
        target = batch[0]["decoder_target"]
        self.val_preds.append(preds.detach().cpu())
        self.val_targets.append(target.detach().cpu())
        self.log("val_loss", loss)
        return loss

    def on_validation_epoch_end(self):
        if self.val_preds and self.val_targets:
            y_true = torch.cat(self.val_targets).numpy()
            y_pred = torch.cat(self.val_preds).numpy()
            y_true = np.expm1(y_true.flatten())
            y_pred = np.expm1(y_pred.flatten())

            mae = mean_absolute_error(y_true, y_pred)
            rmse = math.sqrt(mean_squared_error(y_true, y_pred))
            r2 = r2_score(y_true, y_pred)
            smape = 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

            self.log_dict({
                "val_mae": mae,
                "val_rmse": rmse,
                "val_r2": r2,
                "val_smape": smape
            })

        self.val_preds.clear()
        self.val_targets.clear()

    def configure_optimizers(self):
        return self.model.configure_optimizers()

    def forward(self, *args, **kwargs):
        return self.model(*args, **kwargs)

# 🎯 Instantiate raw TFT model
seed_everything(42)

tft_raw = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.03,
    hidden_size=32,
    attention_head_size=1,
    dropout=0.1,
    hidden_continuous_size=16,
    output_size=1,
    loss=SMAPE(),
    logging_metrics=None,
)

# 🛠️ Patch get_attention_mask
tft_raw._encoder_length = max_encoder_length
tft_raw._prediction_length = max_prediction_length

def patched_get_attention_mask(self, encoder_lengths, decoder_lengths):
    encoder_mask = torch.arange(self._encoder_length, device=encoder_lengths.device)[None, :] < encoder_lengths[:, None]
    decoder_mask = torch.arange(self._prediction_length, device=decoder_lengths.device)[None, :] < decoder_lengths[:, None]
    return torch.cat((encoder_mask, decoder_mask), dim=1)

tft_raw.get_attention_mask = types.MethodType(patched_get_attention_mask, tft_raw)

# ✅ Wrap model for Lightning
tft = TFTLightningWrapper(tft_raw)

# ⚡ Trainer
trainer = Trainer(
    max_epochs=30,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices=1 ,
    gradient_clip_val=0.1,
    log_every_n_steps=1,
    enable_progress_bar=True
)

# 🔁 Train
trainer.fit(tft, train_dataloader, val_dataloader if val_dataloader else None)


INFO:lightning_fabric.utilities.seed:Seed set to 42


⚠️ Validation set is empty after filtering. Proceeding without validation.


/usr/local/lib/python3.11/dist-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/pyto

Training: |          | 0/? [00:00<?, ?it/s]

RuntimeError: TemporalFusionTransformer is not attached to a `Trainer`.

In [ ]:
!pip install pytorch-forecasting


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.7/197.7 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
import numpy as np
import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import RMSE
import warnings
warnings.filterwarnings("ignore")

# Load data
df = pd.read_csv("/content/updated_scor_raw_df (22).csv")

# Aggregate and clean
agg_df = df.groupby(['SiteID', 'Year']).agg({
    'scor_lta': 'mean',
    'TempC': 'mean',
    'density': 'mean',
    'LTA_cm2': 'mean'
}).reset_index()

agg_df.rename(columns={"SiteID": "group", "Year": "time_idx"}, inplace=True)
agg_df["group"] = agg_df["group"].astype(str)
agg_df["time_idx"] = agg_df["time_idx"] - agg_df["time_idx"].min()

# Fill missing
for col in ['TempC', 'density', 'LTA_cm2']:
    agg_df[col] = agg_df.groupby('group')[col].transform(lambda x: x.fillna(x.mean()))
    agg_df[col] = agg_df[col].fillna(agg_df[col].mean())

# Params
max_prediction_length = 3
max_encoder_length = 5
training_cutoff = agg_df["time_idx"].max() - max_prediction_length

# TimeSeriesDataSet
training = TimeSeriesDataSet(
    agg_df[agg_df.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="scor_lta",
    group_ids=["group"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    static_categoricals=["group"],
    static_reals=[],
    time_varying_known_reals=["time_idx"],
    time_varying_unknown_reals=["scor_lta", "TempC", "density", "LTA_cm2"],
    target_normalizer=GroupNormalizer(groups=["group"]),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True,
)

# Validation
validation = TimeSeriesDataSet.from_dataset(training, agg_df, predict=True, stop_randomization=True)

# Dataloaders
batch_size = 64
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 2, num_workers=0)

# Trainer setup
pl.seed_everything(42)
early_stop_callback = EarlyStopping(monitor="val_loss", patience=10, mode="min")
lr_logger = LearningRateMonitor()
logger = TensorBoardLogger("lightning_logs")

trainer = pl.Trainer(
    max_epochs=30,
    gpus=1 if torch.cuda.is_available() else 0,
    gradient_clip_val=0.1,
    callbacks=[early_stop_callback, lr_logger],
    logger=logger
)

# Model
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=1e-3,
    hidden_size=16,
    attention_head_size=1,
    dropout=0.1,
    hidden_continuous_size=8,
    output_size=1,
    loss=RMSE(),
    log_interval=10,
    reduce_on_plateau_patience=4,
)

# Train
trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader
)


INFO:lightning_fabric.utilities.seed:Global seed set to 42
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


TypeError: `model` must be a `LightningModule` or `torch._dynamo.OptimizedModule`, got `TemporalFusionTransformer`

In [ ]:
# ✅ Full clean environment
!pip uninstall -y torch torchvision torchaudio pytorch-lightning pytorch-forecasting
!pip install torch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cpu
!pip install pytorch-lightning==1.9.5
!pip install pytorch-forecasting==0.10.0  # ✅ this version works with Lightning 1.9.5 AND Python 3.10


Found existing installation: torch 1.13.1+cpu
Uninstalling torch-1.13.1+cpu:
  Successfully uninstalled torch-1.13.1+cpu
Found existing installation: torchvision 0.14.1+cpu
Uninstalling torchvision-0.14.1+cpu:
  Successfully uninstalled torchvision-0.14.1+cpu
Found existing installation: torchaudio 0.13.1+cpu
Uninstalling torchaudio-0.13.1+cpu:
  Successfully uninstalled torchaudio-0.13.1+cpu
Found existing installation: pytorch-lightning 1.9.5
Uninstalling pytorch-lightning-1.9.5:
  Successfully uninstalled pytorch-lightning-1.9.5
Found existing installation: pytorch-forecasting 1.0.0
Uninstalling pytorch-forecasting-1.0.0:
  Successfully uninstalled pytorch-forecasting-1.0.0
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu
  Using cached https://download.pytorch.org/whl/cpu/torch-1.13.1%2Bcpu-cp310-cp310-linux_x86_64.whl (199.1 MB)
  Using cached https://download.pytorch.org/whl/cpu/torchvision-0.14.1%2Bcpu-cp310-cp310-linux_x86_64.whl (16.8 MB)
  Usi

  Using cached optuna-2.10.1-py3-none-any.whl.metadata (15 kB)
INFO: pip is looking at multiple versions of torchmetrics to determine which version is compatible with other requirements. This could take a while.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 108.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 134.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 153.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.4/891.4 kB 25.5 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.9.0-py3-none-any.whl size=11014 sha256=66104790df2

In [ ]:
# ✅ Fully clean reinstall compatible versions after Python 3.10 switch
!pip install --force-reinstall torch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cpu
!pip install --force-reinstall pytorch-lightning==1.9.5
!pip install --force-reinstall pytorch-forecasting==1.0.0


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu
  Using cached https://download.pytorch.org/whl/cpu/torch-1.13.1%2Bcpu-cp310-cp310-linux_x86_64.whl (199.1 MB)
  Using cached https://download.pytorch.org/whl/cpu/torchvision-0.14.1%2Bcpu-cp310-cp310-linux_x86_64.whl (16.8 MB)
  Using cached https://download.pytorch.org/whl/cpu/torchaudio-0.13.1%2Bcpu-cp310-cp310-linux_x86_64.whl (4.0 MB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached numpy-2.2.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached pillow-11.2.1-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (8.9 kB)
  Using cached charset_normalizer-3.4.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (35 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.4.0-py3-none-any.whl.metadata (6.5 kB)
  Using c

  Using cached pytorch_lightning-1.9.5-py3-none-any.whl.metadata (23 kB)
  Using cached numpy-2.2.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached torch-2.7.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (29 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached PyYAML-6.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached fsspec-2025.3.2-py3-none-any.whl.metadata (11 kB)
  Using cached torchmetrics-1.7.1-py3-none-any.whl.metadata (21 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached lightning_utilities-0.14.3-py3-none-any.whl.metadata (5.6 kB)
^C
^C


In [ ]:
# ✅ Install compatible packages
!pip install torch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cpu
!pip install pytorch-lightning==1.9.5 pytorch-forecasting==1.0.0
!pip install matplotlib pandas scikit-learn


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 105.8 MB/s eta 0:00:00


  Using cached pytorch_lightning-1.9.5-py3-none-any.whl.metadata (23 kB)
  Using cached torchmetrics-1.7.1-py3-none-any.whl.metadata (21 kB)
  Using cached lightning_utilities-0.14.3-py3-none-any.whl.metadata (5.6 kB)
  Using cached lightning-2.5.1-py3-none-any.whl.metadata (39 kB)
Using cached pytorch_lightning-1.9.5-py3-none-any.whl (829 kB)
Using cached lightning-2.5.1-py3-none-any.whl (818 kB)
Using cached lightning_utilities-0.14.3-py3-none-any.whl (28 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 120.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 751.2/751.2 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 117.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 